#  Entregable 4 - Aprendizaje Automático II


***<p style="text-align:center;">Implementación de una MLP con PyTorch</p>***

En este cuaderno vamos a implementar una MLP (Multi-Layer Perceptron), esta vez, ayudándonos de `PyTorch`.

### Normas de Entrega

1. El formato de entrega será en una carpeta comprimida con nombre: {Iniciales de Nombre y Apellidos}_E4.zip, en Aula Virtual en la fecha señalada en la plataforma y comunicada en clase previamente.
    * Por ejemplo: Iván Ramírez Díaz ==> `IRD_E4.zip`
2. El contenido de dicha carpeta será:
    * Obligatorio: Notebook relleno del Entregable 4.
    * Opcional: Memoria (pdf), en caso de necesitar dar alguna explicación.
3. Antes de la entrega, se debe comprobar que el código completo funciona.
4. La entrega es individual.

### Evaluación

La práctica entregable tiene un peso global de 1/4 puntos (los 4 entregables son el 10% de la nota final).

La práctica entregable se calificará sobre 10 puntos. Las puntuaciones son las siguientes:

- **[Ejercicio 1]** Implementa y entrena el modelo `MLP` usando `PyTorch`. (3 puntos)
- **[Ejercicio 2]** Implementa y entrena el modelo `MLP` usando `Sequential` de `PyTorch`. (1 punto)
- **[Ejercicio 3]** Entrena haciendo uso de `GPU` o `TPU` y compara con el tiempo de entrenamiento en `CPU`. (2 puntos)
- **[Ejercicio 4]** Crea un dataset de `PyTorch` (`Dataset`) a partir de los datos de `breast cancer dataset` y entrena una MLP usando la entropía cruzada binaria. (2 puntos)
- **[Ejercicio 5 (para nota)]** Repite todo el proceso ahora para el `Iris` dataset multi-clase. (1 punto)


## **[Ejercicio 1]**

Con los mismos datos del Entregable 3, que se proporcionan a continuación:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(1)

# Parámetros de entrada y salida
input_dim = 2   # Dimensiones de entrada
output_dim = 1  # Dimensiones de salida

# Generación de datos de ejemplo
n_samples = 250
X = np.random.rand(n_samples, input_dim)
y = np.sin(2 * np.pi * X[:,0:1]) + np.cos(2 * np.pi * X[:,1:]) + 0.1 * np.random.randn(n_samples, output_dim)

Haz un split de `train` (70%), `valid` (15%) y `test` (15%):

In [ ]:
######################### COMPLETAR ############################
from sklearn.model_selection import train_test_split

# Dividimos primero en entrenamiento y conjunto temporal de validación+test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Dividimos el conjunto temporal en validación y test
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape[0])
print("Tamaño del conjunto de validación:", X_valid.shape[0])
print("Tamaño del conjunto de prueba:", X_test.shape[0])
###################### FIN COMPLETAR ###########################

Tamaño del conjunto de entrenamiento: 175
Tamaño del conjunto de validación: 37
Tamaño del conjunto de prueba: 38


Crea un modelo MLP en `PyTorch` de forma similar a como lo hiciste en el Entregable 3:

In [ ]:
########################### COMPLETAR ###########################
# Importa lo que necesites
import torch
from torch import nn
from torch.optim import SGD

######################### FIN COMPLETAR #########################

# Definir el modelo MLP
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        ########################### COMPLETAR ###########################
        F1 =   nn.Linear(input_dim, 128)# Capa lineal con 128 neuronas de salida
        Act1 = nn.Sigmoid()  # Capa de activación
        F2 =  nn.Linear(128, 64) # Capa lineal con 64 neuronas de salida
        Act2 = nn.Sigmoid()  # Capa de activación
        F3 = nn.Linear(64, output_dim)  # Capa de salida
        layers_list = [F1, Act1, F2, Act2, F3]
        self.nn_layers = nn.ModuleList(layers_list)

        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        for layer in self.nn_layers:
          x= layer(x)
        return x
        ######################### FIN COMPLETAR #########################

# Inicializar el modelo
input_dim = 2  # Dimensión de entrada
output_dim = 1  # Dimensión de salida

model = MLP(input_dim, output_dim)
print(model)

MLP(
  (nn_layers): ModuleList(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


Entrena el modelo con datos anteriores:

In [ ]:
########################### COMPLETAR ###########################
# Convertir datos a tensores de PyTorch

X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_valid_tensor = torch.Tensor(X_valid)
y_valid_tensor = torch.Tensor(y_valid)

# Inicializar el modelo
model = MLP(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()   # Pérdida para regresión
opt = SGD(model.parameters(), lr=0.1)   # Optimizador

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 24

# Entrenamiento

for epoch in range(n_epochs):
    model.train()  # Modo entrenamiento
    permutation = torch.randperm(X_train_tensor.size(0))  # Mezclar datos

    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]

        ########################### COMPLETAR ###########################
        # Selecciona los batches aleatoriamente con `indices` dado
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Forward pass
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)

        # Backward pass y optimización

        opt.zero_grad()  # Limpiar gradientes acumulados
        loss.backward()        # Calcular gradientes
        opt.step()       # Actualizar parámetros


        ######################### FIN COMPLETAR #########################

    ########################### COMPLETAR ###########################
    # Validación cada 500 épocas
    if (epoch + 1) % 500 == 0:
        # Modo evaluación
        model.eval()
        with torch.no_grad():
            val_predictions = model(X_valid_tensor)
            val_loss = loss_fn(val_predictions, y_valid_tensor)
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    ######################### FIN COMPLETAR #########################


Epoch 500/5000, Loss: 0.4732, Val Loss: 0.6734
Epoch 1000/5000, Loss: 0.1582, Val Loss: 0.2284
Epoch 1500/5000, Loss: 0.0112, Val Loss: 0.0454
Epoch 2000/5000, Loss: 0.0514, Val Loss: 0.0445
Epoch 2500/5000, Loss: 0.0308, Val Loss: 0.0792
Epoch 3000/5000, Loss: 0.0549, Val Loss: 0.0644
Epoch 3500/5000, Loss: 0.0105, Val Loss: 0.0428
Epoch 4000/5000, Loss: 0.0119, Val Loss: 0.0177
Epoch 4500/5000, Loss: 0.0208, Val Loss: 0.0186
Epoch 5000/5000, Loss: 0.0237, Val Loss: 0.0126


Una vez entrenado, podemos comprobar el nivel de ajuste con los datos de entrenamiento:

In [ ]:
import plotly.graph_objs as go

# Prediciones de entrenamiento
predictions = model(X_train_tensor).detach().numpy()

# Plots
fig = go.Figure()

# Plot predicciones
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=predictions[:, 0],
    mode='markers',
    marker=dict(color='red', size=8, symbol='circle', opacity=0.8),
    name='y_pred',
))

# Plot y_train
fig.add_trace(go.Scatter3d(
    x=X_train[:, 0],
    y=X_train[:, 1],
    z=y_train.flatten(),
    mode='markers',
    marker=dict(color='blue', size=8, symbol='circle', opacity=0.8),
    name='y_true',
))

# Ajuste del layout
fig.update_layout(
    scene=dict(
        xaxis_title='X1',
        yaxis_title='X2',
        aspectmode='cube',  # Ensures equal scaling of all axes
        xaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        yaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True),  # Light gray background
        zaxis=dict(backgroundcolor='rgb(230, 230, 230)', gridcolor='white', showbackground=True)   # Light gray background
    ),
    width=800,  # Increase the size of the figure
    height=800,  # Increase the size of the figure
    margin=dict(l=0, r=0, b=0, t=0),  # Reduce margins
    title="Predición de la MLP",  # Add title
    title_font=dict(size=20, family='Arial, sans-serif'),  # Customize title font
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),  # Set the default camera angle for better visualization
)

fig.show()

## **[Ejercicio 2]**

Repite el proceso, ahora, reescribiendo la clase `MLP` (llámala `MLPSeq`) haciendo uso de `Sequential`.

Ayúdate de la documentación: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definir el modelo MLP
class MLPSeq(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPSeq, self).__init__()
        ########################### COMPLETAR ###########################
        self.model= nn.Sequential(nn.Linear(input_dim, 128),
                                  nn.Sigmoid(),
                                  nn.Linear(128, 64),
                                  nn.Sigmoid(),
                                  nn.Linear(64, output_dim))


        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        return self.model(x)
        ######################### FIN COMPLETAR #########################

# Inicializar el modelo
input_dim = 2  # Dimensión de entrada
output_dim = 1  # Dimensión de salida

model = MLPSeq(input_dim, output_dim)
print(model)

MLPSeq(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


Entrena de nuevo y compara los resultados:

In [ ]:
########################### COMPLETAR ###########################
# Entrena de nuevo con MLPSeq y compara los resultados:

# Convertir datos a tensores de PyTorch

X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_valid_tensor = torch.Tensor(X_valid)
y_valid_tensor = torch.Tensor(y_valid)

# Inicializar el modelo
model = MLPSeq(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()  # Pérdida para regresión
optimizer =  optim.SGD(model.parameters(), lr=0.1) # Optimizador

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 64

# Entrenamiento

for epoch in range(n_epochs):
    model.train()  # Modo entrenamiento
    permutation = torch.randperm(X_train_tensor.size(0))  # Mezclar datos

    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]

        # Selecciona los batches aleatoriamente con `indices` dado
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Forward pass
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)

        # Backward pass y optimización

        optimizer.zero_grad()  # Limpiar gradientes acumulados
        loss.backward()        # Calcular gradientes
        optimizer.step()       # Actualizar parámetros


        ######################### FIN COMPLETAR #########################

    ########################### COMPLETAR ###########################
    # Validación cada 500 épocas
    if (epoch + 1) % 500 == 0:
        # Modo evaluación
        model.eval()
        with torch.no_grad():
            val_predictions = model(X_valid_tensor)
            val_loss = loss_fn(val_predictions, y_valid_tensor)

        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    ######################### FIN COMPLETAR #########################


Epoch 500/5000, Loss: 0.7836, Val Loss: 0.7304
Epoch 1000/5000, Loss: 0.6316, Val Loss: 0.7214
Epoch 1500/5000, Loss: 0.6172, Val Loss: 0.5854
Epoch 2000/5000, Loss: 0.4082, Val Loss: 0.4556
Epoch 2500/5000, Loss: 0.3264, Val Loss: 0.3115
Epoch 3000/5000, Loss: 0.0622, Val Loss: 0.0880
Epoch 3500/5000, Loss: 0.0402, Val Loss: 0.0588
Epoch 4000/5000, Loss: 0.0381, Val Loss: 0.0377
Epoch 4500/5000, Loss: 0.0422, Val Loss: 0.0352
Epoch 5000/5000, Loss: 0.0254, Val Loss: 0.0310


## **[Ejercicio 3]**

Para comparar los tiempos de entrenamiento, primero vamos a implementar el entrenamiento como función.

Define una función `train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)`

In [ ]:
def train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size):
    ########################### COMPLETAR ###########################

      for epoch in range(n_epochs):
          model.train()  # Modo entrenamiento
          permutation = torch.randperm(X_train_tensor.size(0))  # Mezclar datos

          for i in range(0, X_train_tensor.size(0), batch_size):
              indices = permutation[i:i + batch_size]

              # Selecciona los batches aleatoriamente con `indices` dado
              batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

              # Forward pass
              predictions = model(batch_X)
              loss = loss_fn(predictions, batch_y)

              # Backward pass y optimización

              optimizer.zero_grad()  # Limpiar gradientes acumulados
              loss.backward()        # Calcular gradientes
              optimizer.step()       # Actualizar parámetros

    ######################### FIN COMPLETAR #########################


Entrena el mismo modelo en `CPU` y en `GPU` para distintos valores del batch. ¿Qué conclusiones sacas?

In [ ]:
X_train_tensor.device

device(type='cpu')

In [ ]:
import time

# Convertir datos a tensores de PyTorch
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_valid_tensor = torch.Tensor(X_valid)
y_valid_tensor = torch.Tensor(y_valid)

# Inicializar el modelo
model = MLPSeq(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()  # Pérdida para regresión
optimizer =  optim.SGD(model.parameters(), lr=0.01) # Optimizador

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 64
start = time.time()

train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)
######################### FIN COMPLETAR #########################

end = time.time()
print('CPU time: {}'.format(end - start))


CPU time: 13.385776281356812


In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Wed Dec  4 15:46:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
########################### COMPLETAR ###########################
# Convertir datos a tensores de PyTorch y pasarlos a GPU
device = 'cuda'


X_train_tensor = (torch.Tensor(X_train)).to(device)
y_train_tensor = (torch.Tensor(y_train)).to(device)
X_valid_tensor = (torch.Tensor(X_valid)).to(device)
y_valid_tensor = (torch.Tensor(y_valid)).to(device)

# Inicializar el modelo
model = (MLPSeq(input_dim, output_dim)).to(device)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()  # Pérdida para regresión
optimizer =  optim.SGD(model.parameters(), lr=0.01) # Optimizador

######################### FIN COMPLETAR #########################

# Configuración del entrenamiento
n_epochs = 5000
batch_size = 64
start = time.time()

train(model, X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, loss_fn, optimizer, n_epochs, batch_size)

######################### FIN COMPLETAR #########################

end = time.time()
print('GPU time: {}'.format(end - start))

GPU time: 19.256194353103638


Con la configuración inicial de las capas se obtienenen tiempos mayores en GPU que en CPU, cuya diferencia disminuye al aumentar el tamaño del batch.

```

#Configuración inicial
model= nn.Sequential(nn.Linear(input_dim, 128),
                                  nn.Sigmoid(),
                                  nn.Linear(128, 64),
                                  nn.Sigmoid(),
                                  nn.Linear(64, output_dim))

Tiempos para la configuración inicial:
------------------------------------------
Batch Size  Tiempo CPU (s)  Tiempo GPU (s)
------------------------------------------
32            25.39           39.58
64            13.39           19.26
128           11.35           13.91

```


Si aumentamos el número de neuronas en las capas ocultas de la red sí se puede paralelizar y aprovechar la eficiencia de la GPU obteniendo para batches más grandes un mejor rendimiento en GPU:

```

#Configuración 2 (más neuronas)
model= nn.Secuential(nn.Linear(input_dim, 512),
                                  nn.Sigmoid(),
                                  nn.Linear(512, 256),
                                  nn.Sigmoid(),
                                  nn.Linear(256, output_dim))


Tiempos para la configuración 2:
------------------------------------------
Batch Size  Tiempo CPU (s)  Tiempo GPU (s)
------------------------------------------
32            41.88           42.42
64            30.51           22.80
128           25.25           14.16
------------------------------------------
```






## **[Ejercicio 4]**

Crea, a partir del `breast_cancer_dataset` que ya conoces, un dataset de `PyTorch`. Crea su correspondiente dataloader para train, valid, y test:

In [ ]:
class MLP4(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP4, self).__init__()
        ########################### COMPLETAR ###########################
        self.model= nn.Sequential(nn.Linear(input_dim, 512),
                                  nn.Sigmoid(),
                                  nn.Linear(512, 256),
                                  nn.Sigmoid(),
                                  nn.Linear(256, output_dim),
                                  nn.Sigmoid())


        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        return self.model(x)
        ######################### FIN COMPLETAR #########################

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

torch.set_default_dtype(torch.float32)

# Carga datos y etiquetas
cancer = load_breast_cancer()
data, labels = cancer.data, cancer.target

# Crea el dataset
class BreastCancerDataset(Dataset):
    ########################### COMPLETAR ###########################
    def __init__(self, data, labels):
        """
        Constructor del Dataset que almacena los datos y las etiquetas.
        """
        self.data = data
        self.labels = labels


    def __len__(self):
        """
        Devuelve el tamaño del Dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Devuelve un dato (features) y su etiqueta correspondiente.
        """
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)


        ######################### FIN COMPLETAR #########################

########################### COMPLETAR ###########################

# Dividir el dataset en train (70%), valid (15%) y test (15%)
X_train, X_temp, y_train, y_temp =train_test_split(data, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test =train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Crear los datasets para train, valid y test
train_dataset = BreastCancerDataset(X_train, y_train)
valid_dataset =BreastCancerDataset(X_valid, y_valid)
test_dataset =BreastCancerDataset(X_test, y_test)

# Crear los DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Para entrenamiento, mezclar los datos
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)  # Para validación, no mezclar los datos
test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # Para test, no mezclar los datos
######################### FIN COMPLETAR #########################

for batch_idx, (batch_data, batch_labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print(f"Datos: {batch_data.shape}")  # Dimensiones del batch de datos
    print(f"Etiquetas: {batch_labels.shape}")  # Dimensiones del batch de etiquetas

Batch 1:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 2:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 3:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 4:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 5:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 6:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 7:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 8:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 9:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 10:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 11:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 12:
Datos: torch.Size([32, 30])
Etiquetas: torch.Size([32])
Batch 13:
Datos: torch.Size([14, 30])
Etiquetas: torch.Size([14])


Crea una función de entrenamiento `train_model` y otra de evaluación `evaluate_model`:

In [ ]:


# Función de entrenamiento y validación
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs, device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a device
    # Completar
    model.to(device)

    for epoch in range(n_epochs):
        # Modelo en modo entrenamiento
        # Cpmpletar
        model.train()
        train_loss = 0.0
        for features, labels in train_loader:
            # Enviar datos a 'device
            features = features.to(device)
            labels = labels.to(device).unsqueeze(1)  # Ajustar dimensiones y enviar a GPU

            # Forward
            outputs = model(features)
            loss = loss_fn(outputs, labels)

            parameter = model.parameters()

            # Backward y optimización
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            train_loss += loss.item() * features.size(0)

        train_loss /= len(train_loader.dataset)

        # Modelo en modo validación
        # Completar

        valid_loss = 0.0
        with torch.no_grad():
            model.eval()
            for features, labels in valid_loader:
                features = features.to(device)
                labels = labels.to(device).unsqueeze(1)
                outputs = model(features)
                loss = loss_fn(outputs, labels)
                valid_loss += loss.item() * features.size(0)
        valid_loss /= len(valid_loader.dataset)

        print(f"Epoch {epoch + 1}/{n_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
    ######################### FIN COMPLETAR #########################


# Función para evaluar el modelo
def evaluate_model(model, test_loader, device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a device
    # Completar
    model.to(device)
    # Modelo en modo validación
    # Completar
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # No hace falta calcular gradientes en evaluación. Así, evitamos cómputos innecesarios
        for features, labels in test_loader:
            # Enviar datos a 'device
            features = features.to(device)
            labels = labels.to(device).unsqueeze(1)
            # Predicciones
            outputs = model(features)
            predictions = (outputs > 0.5) #Umbral de clasificación
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

Entrena y evalúa:

In [ ]:
########################### COMPLETAR ###########################

# Inicializar el modelo
# Parámetros de entrada y salida
input_dim = data.shape[1]   # Dimensiones de entrada
output_dim = 1  # Dimensiones de salida

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Detectar GPU

model =MLP4(input_dim, output_dim)

# Definir función de pérdida y optimizador
loss_fn = nn.BCELoss()  # Ojo con la última capa de activación del modelo
optimizer = SGD(model.parameters(), lr=0.01)

n_epochs = 50

# Entrenar el modelo
train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs, device)

# Evaluar el modelo
evaluate_model(model, test_loader,device)

######################### FIN COMPLETAR #########################


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## **[Ejercicio 5 (para nota)]**

Entrena un modelo MLP para el nuevo dataset `Iris`.

Para ello, primero crea un `Dataset` y crea también sus dataloaders:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Verificar si CUDA está disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cargar el conjunto de datos Iris
iris=  load_iris()
data, labels = iris.data, iris.target

# Convertir a tensores
class IrisDataset(Dataset):
    def __init__(self, data, labels):
        """
        Constructor que toma un dataset y lo convierte a tensores.
        """
        self.data= data
        self.labels=labels


    def __len__(self):
      return len(self.data)



    def __getitem__(self, idx):
       return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)


# Dividir el dataset en entrenamiento (25%), validación (25%) y prueba (50%)
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.5,random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5,random_state=42)

# Crear los DataLoaders
batch_size = 32

train_ds = IrisDataset(X_train, y_train)
valid_ds = IrisDataset(X_valid, y_valid)
test_ds = IrisDataset(X_test, y_test)

train_loader =DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader =DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
test_loader =DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Para confirmar la división
print(f"Training set size: {len(train_loader.dataset)}")
print(f"Validation set size: {len(valid_loader.dataset)}")
print(f"Test set size: {len(test_loader.dataset)}")

Training set size: 75
Validation set size: 37
Test set size: 38


Crea el modelo MLP que consideres:

In [ ]:
# Definir el modelo MLP
class MLP5(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP5, self).__init__()
        ########################### COMPLETAR ###########################
        self.model= nn.Sequential(nn.Linear(input_dim, 128),
                                  nn.ReLU(),
                                  nn.Linear(128, 64),
                                  nn.ReLU(),
                                  nn.Linear(64, output_dim))
                                  #nn.Softmax()) No hace falta porque usaremos CrossEntropyLoss

        ######################### FIN COMPLETAR #########################

    def forward(self, x):
        ########################### COMPLETAR ###########################
        return self.model(x)
        ######################### FIN COMPLETAR #########################


Crea (adapta) las funciones de `train_model` y `evaluate_model`:

In [ ]:
# Función de entrenamiento y validación
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs=n_epochs, device=device):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
        # Completar
    model.to(device)
    for epoch in range(n_epochs):
        # Modelo en modo entrenamiento
        model.train()
           # Completar

        train_loss = 0.0
        for features, labels in train_loader:

            # Enviar datos a 'device
            features = features.to(device)
            labels = labels.to(device).long()

            # Forward
            outputs = model(features)
            loss = loss_fn(outputs, labels)

            # Backward y optimización
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * features.size(0)

        train_loss /= len(train_loader.dataset)

        # Modelo en modo validación
        model.eval()
            # Completar
        valid_loss = 0.0
        with torch.no_grad():
            for features, labels in valid_loader:
                features = features.to(device)
                labels = labels.to(device).long()
                outputs = model(features)
                loss = loss_fn(outputs, labels)
                valid_loss += loss.item() * features.size(0)
        valid_loss /= len(valid_loader.dataset)
        if epoch % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
    ######################### FIN COMPLETAR #########################


def evaluate_model(model, test_loader, device=device, name=None):
    ########################### COMPLETAR ###########################
    # Traslada el modelo a `device`
    model.to(device)
        # Completar

    # Modelo en modo validación
    model.eval()
        # Completar

    correct = 0
    total = 0
    with torch.no_grad():  # No hace falta calcular gradientes en evaluación
        for features, labels in test_loader:
            # Enviar datos a 'device'
                features = features.to(device)
                labels = labels.to(device).long()

            # Predicciones
                probabilities =model(features)
                #probabilities = torch.softmax(outputs, dim=1)  # Ojo con la última función del modelo MLP

                # Obtener las clases predichas (devuelve la clase con mayor probabilidad)
                predicted =torch.argmax(probabilities, dim=1)

                # Contabilizar las predicciones correctas
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

    accuracy = correct / total
    print(f"{name} Accuracy: {accuracy:.4f}")

    ######################### FIN COMPLETAR #########################


Entrena y valida el modelo:

In [ ]:
########################### COMPLETAR ###########################
# Inicializar el modelo
input_dim =data.shape[1]
output_dim = len(np.unique(labels))
model = MLP5(input_dim, output_dim)

# Definir la función de pérdida y el optimizador
loss_fn = nn.CrossEntropyLoss()  #  clasificación multiclase
optimizer = SGD(model.parameters(), lr=0.1)

n_epochs = 500  # Definir el número de épocas de entrenamiento

######################### FIN COMPLETAR #########################

train_model(model, train_loader, valid_loader, loss_fn, optimizer, n_epochs=n_epochs, device=device)

evaluate_model(model, train_loader, device=device, name='Train')
evaluate_model(model, valid_loader, device=device, name='Valid')
evaluate_model(model, test_loader, device=device, name='Test')

Epoch 1/500, Train Loss: 1.1513, Valid Loss: 1.3195
Epoch 51/500, Train Loss: 0.9692, Valid Loss: 0.5637
Epoch 101/500, Train Loss: 0.2407, Valid Loss: 0.1452
Epoch 151/500, Train Loss: 0.0981, Valid Loss: 0.0737
Epoch 201/500, Train Loss: 0.0920, Valid Loss: 0.0871
Epoch 251/500, Train Loss: 0.1083, Valid Loss: 0.1831
Epoch 301/500, Train Loss: 0.1109, Valid Loss: 0.1125
Epoch 351/500, Train Loss: 0.1246, Valid Loss: 0.0865
Epoch 401/500, Train Loss: 0.1058, Valid Loss: 0.7604
Epoch 451/500, Train Loss: 0.1143, Valid Loss: 0.2056
Train Accuracy: 0.9600
Valid Accuracy: 0.8649
Test Accuracy: 0.9737
